### Final Data Preparation (Silver to Gold)
This step ensures the dataset is perfectly typed and saved in a final "Gold" version for the RFM model.

In [2]:
import pandas as pd
import os

# Define paths
INPUT_PATH = '../data/2019-oct_chunk.parquet'
GOLD_PATH = '../data/Gold-Data/2019-Oct-Gold.parquet'

# Ensure directory exists
os.makedirs('../data/Gold-Data', exist_ok=True)

# Load the cleaned chunk data
df = pd.read_parquet(INPUT_PATH)

# Final type optimization for memory efficiency
categorical_cols = ['brand', 'category_code', 'event_type']
for col in categorical_cols:
    df[col] = df[col].astype('category')

# Save to final Gold version
print(f"Saving final 'Gold' dataset to: {GOLD_PATH}")
df.to_parquet(GOLD_PATH, index=False, compression='snappy')
print("✨ Gold dataset is ready for analysis.")

Saving final 'Gold' dataset to: ../data/Gold-Data/2019-Oct-Gold.parquet
✨ Gold dataset is ready for analysis.


### RFM Metric Calculation
Here the core metrics are calculated : Recency (days since last purchase),
 Frequency (total unique sessions with a purchase), and Monetary (total spent).

In [3]:
import pandas as pd

# Load only purchase events for RFM calculation
df = pd.read_parquet('../data/Gold-Data/2019-Oct-Gold.parquet')
purchase_df = df[df['event_type'] == 'purchase'].copy()

# Define the snapshot date (one day after the last recorded purchase)
snapshot_date = purchase_df['event_time'].max() + pd.Timedelta(days=1)

# Aggregate data by user_id
rfm = purchase_df.groupby('user_id').agg({
    'event_time': lambda x: (snapshot_date - x.max()).days, # Recency
    'user_session': 'nunique',                            # Frequency
    'price': 'sum'                                         # Monetary
}).reset_index()

# Rename columns for clarity
rfm.columns = ['user_id', 'Recency', 'Frequency', 'Monetary']

print("RFM Metrics preview:")
print(rfm.head())

RFM Metrics preview:
     user_id  Recency  Frequency     Monetary
0  264649825       26          2  1240.040039
1  303160429       19          1   340.589996
2  340041246       18          3   915.520020
3  371877401       23          1    29.889999
4  384989212       14          1    41.439999


###  Advanced Scoring Function (Hybrid Cut & QCut)
This function is the "brain" of scoring system. It allows you to use Business Bins (manual) for Recency and Statistical Bins (quantiles) for Frequency and Monetary.

In [5]:
def calculate_rfm_score(series, n_bins=5, bins=None, reverse=False):
    """
    Calculates an RFM score for a given series.
    - If bins are provided, it uses manual pd.cut (Business Logic).
    - If no bins, it uses pd.qcut with rank to handle duplicates (Statistical Logic).
    """

    if bins is not None:
        # Manual binning for business-defined thresholds
        labels = list(range(n_bins, 0, -1)) if reverse else list(range(1, n_bins + 1))
        return pd.cut(series, bins=bins, labels=labels, include_lowest=True).astype(int)
    else:
        # Quantile-based binning to ensure equal distribution of users
        labels = list(range(n_bins, 0, -1)) if reverse else list(range(1, n_bins + 1))
        return pd.qcut(series.rank(method='first'), q=n_bins, labels=labels).astype(int)

# Apply scoring
# Note: Recency is better with manual bins based on the 31-day data range
r_manual_bins = [0, 7, 14, 21, 28, rfm['Recency'].max() + 1]

rfm['R_Score'] = calculate_rfm_score(rfm['Recency'], n_bins=5, bins=r_manual_bins, reverse=True)
rfm['F_Score'] = calculate_rfm_score(rfm['Frequency'], n_bins=5)
rfm['M_Score'] = calculate_rfm_score(rfm['Monetary'], n_bins=5)

# Calculate final aggregated RFM score
rfm['RFM_Sum_Score'] = rfm[['R_Score', 'F_Score', 'M_Score']].sum(axis=1)
rfm.head()

,user_id,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Sum_Score
0,264649825,26,2,1240.040039,2,4,5,11
1,303160429,19,1,340.589996,3,1,4,8
2,340041246,18,3,915.520020,3,5,5,13
3,371877401,23,1,29.889999,2,1,1,4
4,384989212,14,1,41.439999,4,1,1,6


### Customer Segmentation and Export
Finalizing the analysis by grouping users into actionable segments.

In [6]:
import datetime

# Map the sum score to qualitative segments
# 3-5: Hibernating, 6-9: At Risk, 10-12: Loyal, 13-15: Champions
rfm['Segment'] = pd.cut(
    rfm['RFM_Sum_Score'],
    bins=[0, 5, 9, 12, 15],
    labels=['Hibernating', 'At Risk', 'Loyal', 'Champions']
)

# Display results
print("\nSegment Distribution:")
print(rfm['Segment'].value_counts())

# Save the report with a timestamp
current_date = datetime.datetime.now().strftime('%Y-%m-%d')
report_path = f'../data/Gold-Data/rfm_report_{current_date}.parquet'
rfm.to_parquet(report_path)

print(f"\n✅ Analysis complete. Report saved to: {report_path}")
rfm.head()


Segment Distribution:
Segment
At Risk        154736
Loyal          100265
Champions       63479
Hibernating     28638
Name: count, dtype: int64

✅ Analysis complete. Report saved to: ../data/Gold-Data/rfm_report_2026-01-28.parquet


,user_id,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Sum_Score,Segment
0,264649825,26,2,1240.040039,2,4,5,11,Loyal
1,303160429,19,1,340.589996,3,1,4,8,At Risk
2,340041246,18,3,915.520020,3,5,5,13,Champions
3,371877401,23,1,29.889999,2,1,1,4,Hibernating
4,384989212,14,1,41.439999,4,1,1,6,At Risk
